# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [333]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier
!pip3 install spacy

In [334]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt


In [335]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

iter_indexer = pt.IterDictIndexer("./index", meta={'docno': 50, 'text': 4096}, overwrite=True, blocks=True)



### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [336]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
index = iter_indexer.index(pt_dataset.get_corpus_iter())

topics = pt_dataset.get_topics(variant='title')

query_entity_linking = tira.pt.transform_queries('ir-benchmarks/marcel-gohsen/entity-linking', pt_dataset)

linked_queries = query_entity_linking(topics)


bm25 = pt.BatchRetrieve(index, wmodel="BM25")


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  70%|███████   | 89179/126958 [00:18<00:08, 4423.72it/s]

15:42:14.499 [ForkJoinPool-11-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:24<00:00, 5247.86it/s] 


15:42:23.730 [ForkJoinPool-11-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


In [337]:
# Fragezeichen von den Queries entfernen, sonst gibt es Probleme bei der Transformation
for index, row in linked_queries.iterrows():
    if '?' in str(row['query']):
        linked_queries['query'] = linked_queries['query'].str.replace('?', '')


### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [338]:

queries_entities = linked_queries['entities'].to_dict()
score_threshold = 0.7 # Score für Query Rewriting.
entity_list_size = 2 # Wie viele Entitäten sollen übernommen werden und die Query ersetzen

for qid in queries_entities:
    query_entities = queries_entities[qid] # Eine Liste mit entitäten, die von jeder Query kommt
    if(len(query_entities) > 0): # Manche Query besitzen vielleicht keine Entitäten
        keyphrases = {} # Dict für die Keyphrases

        j = 0;
        for i in range(0,entity_list_size):
            if(entity_list_size < len(query_entities)):
                entity_keyphrase = query_entities[i]['mention']
                entity_score = query_entities[i]['score']
                keyphrases[i] = (entity_keyphrase, entity_score) # keyphrases[i][0] := Entität und keyphrases[i][1] := Der Score der Entität
            else:                                                # Wenn entity_list_size zu groß ist für len(query_entities), dann nehme die maximale mögliche anzahl, also len(query_entities)
                entity_keyphrase = query_entities[i]['mention']
                entity_score = query_entities[i]['score']
                keyphrases[i] = (entity_keyphrase, entity_score)
                j += 1
                if(j == len(query_entities)):
                    break

        delta = 0 # Mittelwert des Scores von den einzelen Entitäten aus einer Query
        for i in range(0,len(keyphrases)):
            delta += keyphrases[i][1]
            delta = delta / len(keyphrases)
    
        result = ""
        if( delta >= score_threshold):
            for i in range(0,len(keyphrases)):
                result += f'"{keyphrases[i][0]}"' + f" " 
            linked_queries['query'][qid] = result


In [339]:
bm25QE = linked_queries >> bm25

# Print or inspect the results
(linked_queries)

/tmp/ipykernel_408/3397989100.py:1: DeprecationWarning: Coercion of a dataframe into a transformer is deprecated; use a pt.Transformer.from_df() instead
  bm25QE = linked_queries >> bm25


,qid,query,original_query,entities
0,1,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
1,2,"""language identification"" ""machine learning""","{'query_id': '2', 'title': 'machine learning l...","[{'begin': 17, 'end': 40, 'mention': 'language..."
2,3,social media detect self-harm,"{'query_id': '3', 'title': 'social media detec...","[{'begin': 0, 'end': 12, 'mention': 'social me..."
3,4,stemming for arabic languages,"{'query_id': '4', 'title': 'stemming for arabi...","[{'begin': 0, 'end': 8, 'mention': 'stemming',..."
4,5,audio based animal recognition,"{'query_id': '5', 'title': 'audio based animal...","[{'begin': 12, 'end': 18, 'mention': 'animal',..."
...,...,...,...,...
63,65,information in different language,"{'query_id': '65', 'title': 'information in di...","[{'begin': 15, 'end': 24, 'mention': 'differen..."
64,66,Abbreviations in queries,"{'query_id': '66', 'title': 'Abbreviations in ...","[{'begin': 0, 'end': 13, 'mention': 'abbreviat..."
65,67,lemmatization algorithms,"{'query_id': '67', 'title': 'lemmatization alg...","[{'begin': 0, 'end': 13, 'mention': 'lemmatiza..."
66,68,filter ad rich documents,"{'query_id': '68', 'title': 'filter ad rich do...","[{'begin': 0, 'end': 6, 'mention': 'filter', '..."


In [340]:
# Experiment ohne Query Expansion

pt.Experiment(
    [bm25QE],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_1000"],
    names=["TF-IDF"]
)
#                                ndcg_cut    recip_rank  recall_1000
# Baseline         	            0.374041    0.579877 	0.825376
# Entitäen 2 Threshold 0.4      0.254208 	0.426573 	0.578069
# Entitäen 2 Threshold 0.7 	    0.331021 	0.509386 	0.711281
# Entitäen 2 Threshold 0.9 	    0.35691 	0.557468 	0.783464
# Entitäen 4 Threshold 0.2 	    0.219243 	0.368677 	0.49455
# Entitäen 2 Threshold 0.2 	    0.221619 	0.370719 	0.49455

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_1000
0,TF-IDF,0.331021,0.509386,0.711281


### Step 4: Create the Run


In [341]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [342]:
print('Now we do the retrieval...')
run = bm25QE(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...
Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query,original_query,entities
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
5,1,82472,1998.sigirconf_conference-98.15,5,13.901647,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
6,1,94415,2008.cikm_conference-2008.183,6,13.808208,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
7,1,17496,O01-2005,7,13.749449,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
8,1,82490,1998.sigirconf_conference-98.33,8,13.735541,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."
9,1,124801,2006.ipm_journal-ir0volumeA42A3.2,9,13.569263,retrieval system improving effectiveness,"{'query_id': '1', 'title': 'retrieval system i...","[{'begin': 27, 'end': 40, 'mention': 'effectiv..."


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [343]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
